Explication 1-1 pas intéréssante

In [9]:
import pandas as pd
from gurobipy import *

# =========================
# 1. Chargement des données
# =========================

df = pd.read_csv("breastcancer_processed.csv")

target = "Benign"
features = [
    "ClumpThickness",
    "UniformityOfCellSize",
    "UniformityOfCellShape",
    "MarginalAdhesion",
    "SingleEpithelialCellSize",
    "BareNuclei",
    "BlandChromatin",
    "NormalNucleoli",
    "Mitoses"
]

X = df[features]
y = df[target]

# =========================
# 2. Construction des scores
#    (corrélation avec Benign)
# =========================

scores = X.apply(lambda col: col.corr(y))
data = scores.to_dict()

print("\nScores des attributs :")
for k, v in data.items():
    print(f"{k:30s}: {v:+.4f}")

# =========================
# 3. Pros / Cons
# =========================

pros = [k for k, v in data.items() if v > 0]
cons = [k for k, v in data.items() if v < 0]

print(f"\nPros ({len(pros)}): {pros}")
print(f"Cons ({len(cons)}): {cons}")

# =========================
# 4. Trade-offs valides (1–1)
# =========================

valid_pairs = [
    (p, q) for p in pros for q in cons
    if data[p] + data[q] > 0
]

print(f"\nNombre de paires valides : {len(valid_pairs)}")

# =========================
# 5. Modèle Gurobi
# =========================

m = Model("BreastCancer_TradeOff_1_1")

# Variables binaires
c = {
    (p, q): m.addVar(vtype=GRB.BINARY, name=f"c_{p}_{q}")
    for (p, q) in valid_pairs
}

m.update()

# =========================
# 6. Contraintes
# =========================

# Chaque Cons est couvert exactement une fois
for q in cons:
    m.addConstr(
        quicksum(c[(p, q)] for p in pros if (p, q) in valid_pairs) == 1,
        name=f"Cover_{q}"
    )

# Chaque Pros utilisé au plus une fois
for p in pros:
    m.addConstr(
        quicksum(c[(p, q)] for q in cons if (p, q) in valid_pairs) <= 1,
        name=f"Unique_{p}"
    )

# =========================
# 7. Objectif : faisabilité
# =========================

m.setObjective(0, GRB.MINIMIZE)
m.params.outputflag = 0
m.optimize()

# =========================
# 8. Résultat
# =========================

if m.status == GRB.OPTIMAL:
    print("\n✔ Explication 1–1 valide trouvée :\n")
    for (p, q) in valid_pairs:
        if c[(p, q)].x > 0.5:
            print(
                f"Trade-off ({p}, {q}) → "
                f"{data[p]:+.3f} + {data[q]:+.3f} = {data[p] + data[q]:+.3f}"
            )

elif m.status == GRB.INFEASIBLE:
    print("\n✘ Aucun trade-off (1–1) possible.")



Scores des attributs :
ClumpThickness                : +0.7148
UniformityOfCellSize          : +0.8208
UniformityOfCellShape         : +0.8219
MarginalAdhesion              : +0.7063
SingleEpithelialCellSize      : +0.6910
BareNuclei                    : +0.8227
BlandChromatin                : +0.7582
NormalNucleoli                : +0.7187
Mitoses                       : +0.4234

Pros (9): ['ClumpThickness', 'UniformityOfCellSize', 'UniformityOfCellShape', 'MarginalAdhesion', 'SingleEpithelialCellSize', 'BareNuclei', 'BlandChromatin', 'NormalNucleoli', 'Mitoses']
Cons (0): []

Nombre de paires valides : 0

✔ Explication 1–1 valide trouvée :



Explication 1-m

In [ ]:
import pandas as pd
from gurobipy import *

# =========================
# 1. Chargement des données
# =========================

df = pd.read_csv("breastcancer_processed.csv")

features = [
    "ClumpThickness",
    "UniformityOfCellSize",
    "UniformityOfCellShape",
    "MarginalAdhesion",
    "SingleEpithelialCellSize",
    "BareNuclei",
    "BlandChromatin",
    "NormalNucleoli",
    "Mitoses"
]

# Classe cible : Malignant = 1
y = 1 - df["Benign"]
X = df[features]

#aeeaeae
# =========================
# 2. Poids (importance globale)
# =========================

weights = X.apply(lambda col: col.corr(y)).to_dict()

# =========================
# 3. Instance à expliquer
# =========================

i = 0  # patient choisi
x = X.iloc[i]
mu = X.mean()

# =========================
# 4. Calcul des deltas
# =========================

deltas = {}
for f in features:
    deltas[f] = weights[f] * (x[f] - mu[f])

pros = [f for f in deltas if deltas[f] > 0]
cons = [f for f in deltas if deltas[f] < 0]

print("Pros :", pros)
print("Cons :", cons)

# =========================
# 5. Modèle Gurobi 1–m
# =========================

m = Model("BreastCancer_Explanation_1_m")

# x[p,c] = 1 si le Pro p couvre le Con c
x_var = m.addVars(pros, cons, vtype=GRB.BINARY, name="x")

m.update()

# =========================
# 6. Contraintes
# =========================

# A. Chaque Con est couvert exactement une fois
for c in cons:
    m.addConstr(quicksum(x_var[p, c] for p in pros) == 1, name=f"Cover_{c}")

# B. Validité des trade-offs (1–m)
epsilon = 1e-6
for p in pros:
    m.addConstr(
        deltas[p] + quicksum(deltas[c] * x_var[p, c] for c in cons) >= epsilon,
        name=f"Valid_{p}"
    )

# =========================
# 7. Résolution
# =========================

m.setObjective(0, GRB.MINIMIZE)
m.params.outputflag = 0
m.optimize()

# =========================
# 8. Analyse
# =========================

print("\n" + "-" * 40)
if m.status == GRB.OPTIMAL:
    print("✔ Explication (1–m) trouvée\n")

    for p in pros:
        covered = [c for c in cons if x_var[p, c].x > 0.5]
        if covered:
            net = deltas[p] + sum(deltas[c] for c in covered)
            print(f"Pro {p} (Δ = {deltas[p]:+.4f})")
            print(f"  couvre {covered}")
            print(f"  bilan net = {net:+.4f}\n")

elif m.status == GRB.INFEASIBLE:
    print("✘ Aucune explication (1–m) possible.")


Pros : ['UniformityOfCellSize', 'UniformityOfCellShape', 'MarginalAdhesion', 'SingleEpithelialCellSize', 'BareNuclei', 'BlandChromatin', 'NormalNucleoli', 'Mitoses']
Cons : ['ClumpThickness']

----------------------------------------
✔ Explication (1–m) trouvée

Pro UniformityOfCellSize (Δ = +1.7654)
  couvre ['ClumpThickness']
  bilan net = +1.3667



Les cons vont dans le sens de la tumeur benigne et les pros dans le sens de la tumeur maligne.
Pour le premier patient, seul le critère 'ClumpThickness' va dans le sens de la tumeur benigne.
Grâce à l'algorithme on trouve que 'UniformityOfCellSize' est suffisament fort pour neutraliser 'ClumpThickness'.

Etant donné que notre solveur cherche n'importe quelle solution faisable, 'UniformityOfCellSize' est le pro le plus fort.
On obtient le choix minimal et rationnel.

On peut reformuler ce résultat comme suit :
Même si l’épaisseur des amas cellulaires est compatible avec une tumeur bénigne, l’uniformité anormale de la taille des cellules est suffisamment marquée pour justifier une prédiction maligne.

Explication m-1

In [1]:
import pandas as pd
from gurobipy import *

# =========================
# 1. Chargement des données
# =========================

df = pd.read_csv("breastcancer_processed.csv")

features = [
    "ClumpThickness",
    "UniformityOfCellSize",
    "UniformityOfCellShape",
    "MarginalAdhesion",
    "SingleEpithelialCellSize",
    "BareNuclei",
    "BlandChromatin",
    "NormalNucleoli",
    "Mitoses"
]

# Classe cible : Malignant = 1
y = 1 - df["Benign"]
X = df[features]

# =========================
# 2. Poids (importance globale)
# =========================

weights = X.apply(lambda col: col.corr(y)).to_dict()

# =========================
# 3. Instance à expliquer
# =========================

i = 0  # patient choisi
x = X.iloc[i]
mu = X.mean()

# =========================
# 4. Calcul des deltas
# =========================

deltas = {f: weights[f] * (x[f] - mu[f]) for f in features}

pros = [f for f in deltas if deltas[f] > 0]
cons = [f for f in deltas if deltas[f] < 0]

print("Pros :", pros)
print("Cons :", cons)
print("Deltas :", deltas)

# =========================
# 5. Modèle Gurobi m–1
# =========================

m = Model("BreastCancer_Explanation_m_1")

# y[p, c] = 1 si le Pro 'p' couvre le Con 'c'
y_var = m.addVars(pros, cons, vtype=GRB.BINARY, name="y")
m.update()

# =========================
# 6. Contraintes m–1
# =========================

# A. Chaque Pro utilisé au plus une fois
for p in pros:
    m.addConstr(quicksum(y_var[p, c] for c in cons) <= 1, name=f"Unique_{p}")

# B. Chaque Con doit être compensé par un ou plusieurs Pros
epsilon = 1e-6
for c in cons:
    m.addConstr(
        deltas[c] + quicksum(deltas[p] * y_var[p, c] for p in pros) >= epsilon,
        name=f"Valid_Group_for_{c}"
    )

# =========================
# 7. Résolution
# =========================

m.setObjective(0, GRB.MINIMIZE)
m.params.outputflag = 0
m.optimize()

# =========================
# 8. Analyse des résultats
# =========================

print("\n" + "-" * 40)
if m.status == GRB.OPTIMAL:
    print("✔ Explication (m–1) trouvée\n")
    for c in cons:
        assigned_pros = [p for p in pros if y_var[p, c].x > 0.5]
        sum_pros = sum(deltas[p] for p in assigned_pros)
        net_value = deltas[c] + sum_pros
        print(f"Argument Contre {c} (Δ={deltas[c]:+.4f}) compensé par : {assigned_pros}")
        print(f"  cumul Pros = {sum_pros:+.4f}, bilan net = {net_value:+.4f}\n")

elif m.status == GRB.INFEASIBLE:
    print("✘ Aucune explication (m–1) possible.")


Pros : ['UniformityOfCellSize', 'UniformityOfCellShape', 'MarginalAdhesion', 'SingleEpithelialCellSize', 'BareNuclei', 'BlandChromatin', 'NormalNucleoli', 'Mitoses']
Cons : ['ClumpThickness']
Deltas : {'ClumpThickness': np.float64(-0.3987334728092868), 'UniformityOfCellSize': np.float64(1.7653840695625356), 'UniformityOfCellShape': np.float64(1.8206749690384447), 'MarginalAdhesion': np.float64(1.2926320195206518), 'SingleEpithelialCellSize': np.float64(0.8528224423288597), 'BareNuclei': np.float64(2.093477931578111), 'BlandChromatin': np.float64(0.3374834210514831), 'NormalNucleoli': np.float64(1.343705371767472), 'Mitoses': np.float64(0.2554327138706111)}
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2763136
Academic license 2763136 - for non-commercial use only - registered to si___@student-cs.fr

----------------------------------------
✔ Explication (m–1) trouvée

Argument Contre ClumpThickness (Δ=-0.3987) compensé par : ['UniformityOfCellSize',

Ici, chaque con doit être compensé par un ou plusieurs pros et chaque pro peut être utilisé au plus une fois.

Le seul con est 'ClumpThickness', tous les autres critères vont servir à le compenser.

La somme des Pros (+9.7616) est largement supérieure au delta négatif (-0.3987), donc la contrainte est satisfaite.

Même si ClumpThickness est contre, l’ensemble des autres variables (pros) compense largement.

Modèle mixte 1-m et m-1